In [ ]:
import cv2 # OpenCV, which we will use for all the core image processing functions 
           # (like filtering and color space conversions)
import numpy as np
import os
from tqdm.notebook import tqdm # A progress bar
import matplotlib.pyplot as plt

# Define the correct input and output folder paths
input_folder = os.path.join('/kaggle/input/', 'trident', 'DATA', 'Icam')
output_folder = os.path.join('/kaggle/working', 'trident', 'DATA', 'Iphy_enh')

# Define the folder for transmission maps
trans_map_folder = os.path.join('/kaggle/working', 'trident', 'DATA', 'classic_phy_mod_trans_map')

# Automatically create the output folders if they don't exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(trans_map_folder, exist_ok=True)

# Check if the input path exists
if not os.path.exists(input_folder):
    print(f"Error: Could not find the input dataset at: {input_folder}")
else:
    print(f"Input dataset found at: {input_folder}")
    print(f"Output folder created at: {os.path.abspath(output_folder)}")
    print(f"Transmission map folder created at: {os.path.abspath(trans_map_folder)}")

In [ ]:
# Helper function for plotting (for final verification) 
def show_image(image, title='Image', cmap_type='viridis'):
    """
    Helper function to display an image in a Jupyter Notebook.
    Converts BGR to RGB for correct color display.
    """
    # Check if the image is color (3 channels) or grayscale (1 channel)
    if len(image.shape) == 3:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(image, cmap=cmap_type)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Step 1 Function 
def get_dark_channel(image, patch_size=15):
    """
    Computes the Dark Channel Prior (DCP) of an image.
    """
    # Find the minimum value across all 3 color channels (R, G, B) for each pixel
    # This creates a 2D grayscale map
    min_channel = image.min(axis=2)

    # Define the 'patch' or 'window' as a kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (patch_size, patch_size))
    
    # Apply the min filter over the 15x15 neighborhood
    # cv2.erode() with this kernel finds the minimum value within the patch
    dark_channel = cv2.erode(min_channel, kernel)
    
    return dark_channel

# Step 2 Function 
def get_background_light(image, dark_channel, percentile=0.001):
    """
    Estimates the background light (B) from the haziest 0.1% of pixels.
    """
    # Get total number of pixels
    num_pixels = dark_channel.size

    # Calculate number of pixels to take for the top 0.1%
    k = int(num_pixels * percentile)

    # Flatten the dark channel to a 1D array
    flat_dark_channel = dark_channel.flatten()
    
    # Find the indices of the brightest pixels (top k values) 
    # np.argsort sorts from low to high, so we take the last 'k' indices
    indices = np.argsort(flat_dark_channel)[-k:]

    # Convert 1D indices back to 2D (row, col) coordinates
    rows, cols = np.unravel_index(indices, dark_channel.shape)
    
    # Get the B, G, R values from the original image at these locations
    hazy_pixels = image[rows, cols]

    # Average these pixels values across each channel (B, G, R)
    background_light = np.mean(hazy_pixels, axis=0)
    
    return background_light

# Step 3 Function 
def get_transmission_map(image, background_light, patch_size=15, omega=0.95):
    """
    Calculates the transmission map t(x).
    """
    # Normalize the image by the background light
    # Using np.clip to avoid division by zero if B is 0 in a channel
    normalized_image = image / np.clip(background_light, 1e-6, 255.0)

    # Get the dark channel of this normalized image
    normalized_dark_channel = get_dark_channel(normalized_image, patch_size)
    
    # Calculate the transmission map using the formula 
    # omega is set to 0.95 to keep some haze
    transmission_map = 1 - (omega * normalized_dark_channel)
    
    return transmission_map

# Step 4 Function 
def recover_radiance(image, background_light, transmission_map, t0=0.1):
    """
    Recovers the scene radiance J(x) using the main dehazing formula.
    """
    # Clamp the transmission map with t0 = 0.1
    clamped_transmission = np.maximum(transmission_map, t0)
    
    # Expand transmission map from 2D (256x256) to 3D (256x256x1)
    # This allows it to be divided from the 3-channel image (256x256x3)
    transmission_3d = clamped_transmission[..., None]

    # Implement the radiance recovery formula
    numerator = image - background_light # (I(x) - B)
    J_partial = numerator / transmission_3d # (I(x) - B)/max(t(x), t0)
    J = J_partial + background_light # ((I(x) - B)/max(t(x), t0)) + B
    
    return J

# Step 5 Functions 
def get_compensation_factor(background_light):
    """
    Calculates the adaptive compensation factor based on the B vector.
    """
    # Get B, G, R values (OpenCV is BGR)
    B_b, B_g, B_r = background_light

    # Calculate average of "problem" colors (G and B)
    avg_bg = (B_g + B_b) / 2

    # Find Red_Ratio
    # Adding 1e-6 to avoid division by zero
    red_ratio = B_r / (avg_bg + 1e-6)

    # Apply the adaptive logic
    if red_ratio < 0.7:
        factor = 0.5 # Very strong blue cast
    elif red_ratio < 0.85:
        factor = 0.4 # Moderate cast
    else:
        factor = 0.3 # Mild cast
        
    return factor

def apply_red_compensation(image, background_light):
    """
    Applies the adaptive red channel compensation.
    """
    # Get the adaptive factor
    compensation_factor = get_compensation_factor(background_light)

    # Get the Red channel (index 2 for BGR)
    # We must work with a float copy to avoid data type errors
    image_float = image.astype(float)
    red_channel = image_float[..., 2]

    # Apply the boost formula
    red_enhanced = red_channel * (1 + compensation_factor)
    
    # Clip the enhanced red channel to not exceed 255
    red_enhanced_clipped = np.clip(red_enhanced, 0, 255)

    # Put the enhanced channel back into the image
    compensated_image = image_float.copy()
    compensated_image[..., 2] = red_enhanced_clipped

    # Convert back to uint8
    return compensated_image.astype(np.uint8)

# Step 6 Function
def apply_clahe(image, clip_limit=2.0, tile_size=(8, 8)):
    """
    Applies CLAHE to the L (Lightness/brightness) channel (0 = black, max = white) of an image.
    
    """
    # Convert the BGR image to Lab color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    
    # Split the Lab image into L, a, and b channels
    # 'L' is Lightness (brightness), 'a' and 'b' are color channels
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Create a CLAHE object 
    # We use the 8x8 grid size 
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
    
    # Apply CLAHE to the L-channel only
    l_channel_enhanced = clahe.apply(l_channel)

    # Merge the enhanced L-channel back with the original a and b channels
    lab_enhanced = cv2.merge((l_channel_enhanced, a_channel, b_channel))
    
    # Convert the Lab image back to BGR
    final_image = cv2.cvtColor(lab_enhanced, cv2.COLOR_Lab2BGR)
    
    return final_image

def plot_on_ax(ax, image, title, cmap_type='viridis'):
    """
    Helper function to plot an image on a specific Matplotlib axis (subplot).
    """
    if len(image.shape) == 3:
        # Convert BGR to RGB
        ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else:
        # Show grayscale
        ax.imshow(image, cmap=cmap_type)
    ax.set_title(title)
    ax.axis('off')

print("All processing functions are defined.")

In [ ]:
# Main Processing Loop 

# SET THE PROCESSING RANGE HERE 
START_NUMBER = 12001
END_NUMBER = 13000
FILE_EXTENSION = '.png'  # Change to '.jpg' or '.jpeg' if needed

print(f"Starting processing from {START_NUMBER}{FILE_EXTENSION} to {END_NUMBER}{FILE_EXTENSION}...")

# Loop through the specified number range, with a progress bar
for i in tqdm(range(START_NUMBER, END_NUMBER + 1), desc='Processing Images'):
    try:
        # Construct the filename and paths
        filename = f"{i}{FILE_EXTENSION}"
        image_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        # Construct the output path for the transmission map
        trans_map_path = os.path.join(trans_map_folder, filename)

        # Check if the input file actually exists
        if not os.path.exists(image_path):
            # If the file doesn't exist, skip it
            continue

        # Load Image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: Could not read {image_path}. Skipping.")
            continue
            
        # Resize and Convert
        input_image = cv2.resize(image, (256, 256))
        input_image_float = input_image.astype(float)
        
        # Run Full Stage 1 Pipeline
        
        # Step 1: Dark Channel
        dark_channel_map = get_dark_channel(input_image_float, patch_size=15)
        
        # Step 2: Background Light
        background_light_B = get_background_light(input_image_float, dark_channel_map)
        
        # Step 3: Transmission Map
        transmission_map_t = get_transmission_map(input_image_float, background_light_B, patch_size=15, omega=0.95)
        
        # Scale the 0.0-1.0 map to 0-255 and save it as an 8-bit image
        transmission_map_image = (transmission_map_t * 255).astype(np.uint8)
        cv2.imwrite(trans_map_path, transmission_map_image)
        
        # Step 4: Recover Radiance
        dehazed_image_J = recover_radiance(input_image_float, background_light_B, transmission_map_t, t0=0.1)
        dehazed_image_J_uint8 = np.clip(dehazed_image_J, 0, 255).astype(np.uint8)
        
        # Step 5: Red Compensation
        compensated_image = apply_red_compensation(dehazed_image_J_uint8, background_light_B)
        
        # Step 6: CLAHE
        final_enhanced_image = apply_clahe(compensated_image, clip_limit=2.0, tile_size=(8, 8))
        
        # Save the Result 
        cv2.imwrite(output_path, final_enhanced_image)
        
    except Exception as error:
        print(f"Error processing {image_path}: {error}")

print("Batch processing complete!")
print(f"All enhanced images are saved in: {os.path.abspath(output_folder)}")
print(f"All transmission maps are saved in: {os.path.abspath(trans_map_folder)}")

In [ ]:
# Cell: Verify one of the results (Step-by-Step) AND Plot Comparison
# (This cell runs the full pipeline and the 1x3 comparison plot at the end)

# SET YOUR TEST NUMBER HERE
TEST_NUMBER = 21252
FILE_EXTENSION = '.png'

filename = f"{TEST_NUMBER}{FILE_EXTENSION}"

# Define BOTH root paths
input_folder = os.path.join('/kaggle/input/', 'trident', 'DATA', 'Icam')
ref_folder = os.path.join('/kaggle/input/', 'trident', 'DATA', 'Iclean') 

original_path = os.path.join(input_folder, filename)
ref_path = os.path.join(ref_folder, filename) 

# Check if BOTH files exist
if os.path.exists(original_path) and os.path.exists(ref_path):
    # Load and Prepare Original Image
    original_test = cv2.imread(original_path)
    original_resized = cv2.resize(original_test, (256, 256))
    original_resized_float = original_resized.astype(float)
    
    # Load and Prepare Reference Image 
    ref_test = cv2.imread(ref_path)
    ref_resized = cv2.resize(ref_test, (256, 256))
    
    print(f"--- Running full pipeline for {filename} ---")

    # Re-run the pipeline step-by-step 
    
    # Step 1: Dark Channel
    dark_channel_map = get_dark_channel(original_resized_float, patch_size=15)
    
    # Step 2: Background Light
    background_light_B = get_background_light(original_resized_float, dark_channel_map)
    print(f"Step 2: Background Light (B) Vector = {background_light_B}")
    
    # Step 3: Transmission Map
    transmission_map_t = get_transmission_map(original_resized_float, background_light_B, patch_size=15, omega=0.95)
    # Convert for display (0.0-1.0 -> 0-255)
    trans_map_display = (transmission_map_t * 255).astype(np.uint8)
    
    # Step 4: Recover Radiance
    dehazed_image_J = recover_radiance(original_resized_float, background_light_B, transmission_map_t, t0=0.1)
    dehazed_image_J_uint8 = np.clip(dehazed_image_J, 0, 255).astype(np.uint8)
    
    # Step 5: Red Compensation
    compensated_image = apply_red_compensation(dehazed_image_J_uint8, background_light_B)
    
    # Step 6: CLAHE
    final_enhanced_image = apply_clahe(compensated_image, clip_limit=2.0, tile_size=(8, 8))
    
    print("--- Plotting all 6 visual steps ---")

    # Create the 2x3 plot
    fig, axes = plt.subplots(2, 3, figsize=(18, 10)) # 2 rows, 3 columns
    
    # Plot 1: Original Image
    plot_on_ax(axes[0, 0], original_resized, title='(0) Original Input')
    
    # Plot 2: Dark Channel Map
    plot_on_ax(axes[0, 1], dark_channel_map, title='(1) Dark Channel Map', cmap_type='gray')
    
    # Plot 3: Transmission Map
    plot_on_ax(axes[0, 2], trans_map_display, title='(3) Transmission Map', cmap_type='gray')
    
    # Plot 4: Dehazed Image 
    plot_on_ax(axes[1, 0], dehazed_image_J_uint8, title='(4) Dehazed (Radiance Recovered)')
    
    # Plot 5: Red Compensated Image 
    plot_on_ax(axes[1, 1], compensated_image, title='(5) Red Compensated')
    
    # Plot 6: Final Enhanced Image 
    plot_on_ax(axes[1, 2], final_enhanced_image, title='(6) Final Enhanced (CLAHE)')
    
    # Add the title and show the figure
    fig.suptitle(f"Step-by-Step Verification for: {filename}", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make room for suptitle
    plt.show() 
    
    
    print(f"--- Plotting Final Comparison for {filename} ---")

    # Create the 1x3 plot
    fig_comp, axes_comp = plt.subplots(1, 3, figsize=(18, 6)) 
    
    # Plot 1: Original Image
    plot_on_ax(axes_comp[0], original_resized, title=f'Input (I_cam: {filename})')
    
    # Plot 2: Classical Output
    plot_on_ax(axes_comp[1], final_enhanced_image, title='Classical Output (Final)')
    
    # Plot 3: Ground Truth
    plot_on_ax(axes_comp[2], ref_resized, title=f'Ground Truth (I_clean: {filename})')
    
    fig_comp.suptitle(f"Classical Method Verification for: {filename}", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) 
    plt.show() 
    
else:
    # Error message if file is missing
    print(f"Error: Could not find '{filename}'.")
    print(f"Please check that it exists in: {original_path} AND {ref_path}")

In [ ]:
# Calculate Classical Model Metrics
# This cell loads the pre-computed classical results from 'output_folder'
# and compares them to the ground truth to get the average SSIM and PSNR.

# Install/Import dependencies
!pip install -q scikit-image tqdm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm.auto import tqdm
import cv2
import numpy as np
import os

# Define the range of test images to evaluate
# This should match the START_NUMBER and END_NUMBER from processing cell
TEST_RANGE_STR = f"{START_NUMBER}-{END_NUMBER}"

# Helper to parse the range 
def parse_range_string(range_str):
    ids = []
    for part in range_str.split(','):
        part = part.strip()
        if '-' in part:
            start, end = part.split('-')
            ids.extend(list(range(int(start), int(end) + 1)))
        else:
            ids.append(int(part))
    return ids

# Main Evaluation Loop 
image_ids = parse_range_string(TEST_RANGE_STR)
ssim_scores = []
psnr_scores = []

print(f"--- Starting Classical Model Evaluation ---")
print(f"Loading {len(image_ids)} images from range {TEST_RANGE_STR}...")

# Wrap the loop with tqdm for a progress bar
for img_id in tqdm(image_ids):
    img_name = f"{img_id}.png"
    
    # Define paths to the final output and the ground truth
    path_pred = os.path.join(output_folder, img_name)
    path_ref = os.path.join(ref_folder, img_name)

    if not os.path.exists(path_pred) or not os.path.exists(path_ref):
        # print(f"Warning: Skipping {img_id}, file not found.")
        continue
        
    try:
        # 1. Load images
        I_pred_bgr = cv2.imread(path_pred)
        I_ref_bgr = cv2.imread(path_ref)

        # 2. Resize (just in case)
        I_pred_resized = cv2.resize(I_pred_bgr, (256, 256))
        I_ref_resized = cv2.resize(I_ref_bgr, (256, 256))
        
        # 3. Calculate Metrics
        val_psnr = psnr(I_ref_resized, I_pred_resized, data_range=255)
        val_ssim = ssim(I_ref_resized, I_pred_resized, multichannel=True, data_range=255, channel_axis=2)
        
        # 4. Store scores
        psnr_scores.append(val_psnr)
        ssim_scores.append(val_ssim)

    except Exception as e:
        print(f"Error processing image {img_id}: {e}")
        
# Print Final Results 
avg_psnr = np.mean(psnr_scores)
avg_ssim = np.mean(ssim_scores)

print("\n--- Classical Model Evaluation Complete ---")
print(f"Evaluated on {len(ssim_scores)} images from '{output_folder}'.")
print(f"Average PSNR: {avg_psnr:.2f} dB")
print(f"Average SSIM: {avg_ssim:.4f}")

In [ ]:
import os

# Define the paths to your output folders
folder_to_zip1 = '/kaggle/working/trident/DATA/Iphy_enh'
folder_to_zip2 = '/kaggle/working/trident/DATA/classic_phy_mod_trans_map'

# Define the names for your zip files
zip_file_name1 = 'Iphy_enh.zip'
zip_file_name2 = 'classic_phy_mod_trans_map.zip'

print(f"Zipping {folder_to_zip1}...")
# Use the 'zip' command: -r (recursive), -q (quiet, less console spam)
# The first path is the output zip file
# The second path is the folder you want to zip
!zip -r -q {zip_file_name1} {folder_to_zip1}

print(f"Zipping {folder_to_zip2}...")
!zip -r -q {zip_file_name2} {folder_to_zip2}

print("--- Zipping Complete! ---")
print(f"Look for '{zip_file_name1}' and '{zip_file_name2}' in the '/kaggle/working/' directory.")